In [1]:
import argparse
import psycopg2
import csv

In [ ]:
def write_to_equity_latency_db(date):
    # Construct the filename of the CSV file to read from
    filename = f"/home/elliott/development/files/equity_cancelack_INCAPNS_2%highest_latencies_{date}.csv"

    # Connect to the database
    conn = psycopg2.connect(
        host="10.7.8.59",
        database="reports",
        user="scalp",
        password="QAtr@de442",
	    port="5433"
    )

    # Open the CSV file
    with open(filename, "r") as f:
        reader = csv.reader(f)
        next(reader)  # Skip the header row

        # Loop through the rows and insert them into the database
        for row in reader:
            avg_difference = row[0]
            count = row[1]
            exchange = row[2]

            cur = conn.cursor()
            cur.execute(
                "INSERT INTO average_latency_cancel (date, exchange, average_difference, count) VALUES (%s, %s, %s, %s)",
                (date, exchange, avg_difference, count)
            )
            conn.commit()

    # Close the database connection
    conn.close()

# Parse the command-line arguments
parser = argparse.ArgumentParser(description="Add latency data to the database")
parser.add_argument("date", help="the date to use for the latency data (format: yyyy-mm-dd)")
args = parser.parse_args()

# Call the write_to_database function with the specified date
write_to_equity_latency_db(args.date)

In [9]:
import csv
import numpy as np
import pandas as pd
from datetime import datetime
import psycopg2
import argparse
import os


def run_queries1(date, target, sender):
    conn = psycopg2.connect(
        host="10.7.8.59",
        database="fixtransactions",
        user="scalp",
        password="QAtr@de442",
        port="5433"
    )

    query1 = f"""
    CREATE TEMP TABLE temp_table1 AS
    SELECT tag11, tag100, tag41, tag49 
    FROM fixmsg 
    WHERE tag52 >= TIMESTAMP '{date}'
    AND tag52 < TIMESTAMP '{date}' + INTERVAL '1 DAY'
    AND tag56 = '{target}'
    and tag49 = '{sender}'
    AND (tag35 = 'AB' or tag35 = 'D');
    """

    query2 = f"""
    CREATE TEMP TABLE temp_table2 AS
    SELECT a.tag11, a.tag52 AS cancel_52, b.tag52 AS cancelack_52, b.tag41, a.tag100
    FROM fixmsg a
    JOIN fixmsg b ON a.tag11 = b.tag41
    WHERE a.tag52 >= TIMESTAMP '{date}'
    AND a.tag52 < TIMESTAMP '{date}' + INTERVAL '1 DAY'
    AND b.tag52 >= TIMESTAMP '{date}'
    AND b.tag52 < TIMESTAMP '{date}' + INTERVAL '1 DAY'
    and a.tag49 = '{sender}'
    and b.tag56 = '{sender}'
    and b.tag49 = '{target}'
    AND b.tag35 = '8'
    AND b.tag39 = '4'
    AND (a.tag35 = 'AB' or a.tag35 = 'D')
    AND a.tag56 = '{target}';
    """

    with conn.cursor() as cursor:
        cursor.execute(query1)
        cursor.execute(query2)

        cursor.execute("SELECT * FROM temp_table2")
        query_result = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]

    conn.close()

    hash_table = {}
    for row in query_result:
        row_dict = {column_names[i]: row[i] for i in range(len(column_names))}
        hash_table[row_dict["tag11"]] = row_dict

    for key in hash_table.keys():
        hash_table[key]['OID'] = hash_table[key].pop('tag11')
        hash_table[key]['reply_OID'] = hash_table[key].pop('tag41')
        hash_table[key]['exchange'] = hash_table[key].pop('tag100')
        hash_table[key]['sent_time'] = hash_table[key].pop('cancel_52')
        hash_table[key]['reply_time'] = hash_table[key].pop('cancelack_52')

    return hash_table

In [10]:
run_queries1('2023-03-23', 'INCAPNS', 'RBTECH3')

{'370040202411707': {'OID': '370040202411707',
  'reply_OID': '370040202411707',
  'exchange': 'PCXOS',
  'sent_time': datetime.datetime(2023, 3, 23, 13, 30, 51, 132000),
  'reply_time': datetime.datetime(2023, 3, 23, 13, 30, 51, 658000)},
 '370040202411708': {'OID': '370040202411708',
  'reply_OID': '370040202411708',
  'exchange': 'PCXOS',
  'sent_time': datetime.datetime(2023, 3, 23, 13, 30, 51, 228000),
  'reply_time': datetime.datetime(2023, 3, 23, 13, 30, 51, 750000)},
 '370040202411709': {'OID': '370040202411709',
  'reply_OID': '370040202411709',
  'exchange': 'PCXOS',
  'sent_time': datetime.datetime(2023, 3, 23, 13, 30, 51, 347000),
  'reply_time': datetime.datetime(2023, 3, 23, 13, 30, 51, 858000)},
 '370040202411711': {'OID': '370040202411711',
  'reply_OID': '370040202411711',
  'exchange': 'PCXOS',
  'sent_time': datetime.datetime(2023, 3, 23, 13, 30, 51, 670000),
  'reply_time': datetime.datetime(2023, 3, 23, 13, 30, 52, 184000)},
 '370040202411712': {'OID': '37004020241